This notebook demonstrates strain mapping using the image affine transform method implemented in pyXem.

# Change Log

30/03/18 Duncan Johnstone - Updated for pyXem framework

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

## Requirements

pyXem 0.5

HyperSpy 1.4

PyMatGen

# Contents

1. <a href='#sim'> Simulate Strain Model Data</a>
2. <a href='#str'> Strain Mapping as Image Affine Transform</a>

Import pyXem and other required libraries

In [13]:
%matplotlib tk
import pyxem as pxm

import numpy as np
import pymatgen as pmg

from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.components.scalable_reference_pattern import ScalableReferencePattern

from scipy.constants import pi

## <a id='loa'></a> 4. Simulate Strain Test Data

Define silicon crystal structure

In [7]:
si = pmg.Element("Si")
lattice = pmg.Lattice.cubic(5.431)
structure = pmg.Structure.from_spacegroup("Fd-3m",lattice, [si], [[0, 0, 0]])
ediff = pxm.DiffractionGenerator(300., 0.025)

Define a series of affine transformations from 0-2%, apply these to the crystal structure and simulate the diffraction in each case.

In [9]:
from pymatgen.transformations.standard_transformations import DeformStructureTransformation

affines = [[[1, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.002, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.004, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.006, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.008, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.01, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.012, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.014, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.016, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.018, 0, 0], [0, 1, 0], [0, 0,  1]],
           [[1.02, 0, 0], [0, 1, 0], [0, 0,  1]]]

data = []
for affine in affines:
    deform = DeformStructureTransformation(affine)
    strained = deform.apply_transformation(structure)
    diff_dat = ediff.calculate_ed_data(strained, 2.5)
    dpi = diff_dat.as_signal(512, 0.02, 2.5)
    data.append(dpi.data)
strain_series = pxm.ElectronDiffraction(data)

#stack multiple versions of tilt series together so that you have repeats of measurements for error analysis
dp = hs.stack((strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series, strain_series))

Plot the test data to see what it looks like.

In [10]:
dp.plot()

## <a id='loa'></a> 5. Perform Strain Mapping

Create a model to the data which comprises a distorted version of a reference (unstrained) diffraction pattern at each probe position.

In [14]:
m = dp.create_model()
ref = ScalableReferencePattern(dp.inav[0,0])
m.append(ref)

Print the affine transform values associated with the distorted diffraction pattern before fitting.

In [15]:
m.print_current_values()

Components	Parameter	Value
ScalableReferencePattern
		d11	1
		d12	0
		d21	0
		d22	1
		t1	0
		t2	0


Perform fitting

In [16]:
m.multifit()

A Jupyter Widget

Construct the displacement graident tensor at each pixel from the fitting results

In [17]:
disp_grad = ref.construct_displacement_gradient()

Perform (right) polar decomposition on the displacement gradient tensor to get rotation matrix, R, and strain matrix, U, at each pixel.

In [18]:
R, U = disp_grad.polar_decomposition()

A Jupyter Widget

Get a strain map and plot it.

In [19]:
strain_map = disp_grad.get_strain_maps()
strain_map.plot()

A Jupyter Widget

A Jupyter Widget